## pos

Here we extract useful pos tags from the title/meta description.

Setting it up

In [1]:
import sys
sys.path.append('../modules/')
from webpage import *
import nlp

#### Looking at meta data
Let's see if we can find something interesting in there

In [2]:
webpages ={ website:Webpage(test_url[website]) for website in test_url}
[page.load() for page in webpages.values()]

c:\anaconda\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
../modules/webpage.py:93: UserWarning: Unable to correctly parse given URL:http://www.nytimes.com/2015/09/11/world/netanyahu-makes-quick-pivot-from-loss-on-iran-deal.html
  warn('Unable to correctly parse given URL:%s' %self.url)


[None, None, None, None, None, None, None, None]

In [3]:
dir(webpages['amazon'])

['__doc__',
 '__goose_article__',
 '__init__',
 '__module__',
 '_valid_text_',
 'html',
 'load',
 'meta_description',
 'meta_keywords',
 'set_url',
 'tags',
 'text',
 'title',
 'url']

In [4]:
[(page.title,page.meta_description,page.meta_keywords) for page in webpages.values()]

[('Georgia Institute of Technology College of Computing - Wikipedia, the free encyclopedia',
  '',
  ''),
 ('F.C. Barcelona News and Discussion  /r/Barca',
  'reddit: the front page of the internet',
  'reddit, reddit.com, vote, comment, submit'),
 ("I'm scared of Louis van Gaal's 'bulldog face', admits Manchester United's Marcos Rojo",
  'Manchester United defender Marcos Rojo admits hes scared of Louis van Gaals bulldog face.',
  ''),
 ('Barca look to continue success against Atleti',
  "<p>Atletico Madrid and Barcelona will meet in an early-season titan clash, one that ESPN FC's Gab Marcotti feels is not a make or break matchup.</p>",
  u'Latest,Atletico Madrid,Spanish Primera Divisi\xf3n,Barcelona,XBOX 360'),
 ('', '', u''),
 ('How to Introduce Your Indoorsy Friend to the Outdoors',
  "Share your passion for the outdoors and introduce a friend to hike and camp--don't forget to start slow.",
  ''),
 ('Amazon.com: Cuisinart CPT-122 Compact 2-Slice Toaster: Kitchen & Dining',
  'Onlin

In [5]:
for tree in nlp.sent_chunker('Amazon.com: Cuisinart CPT-122 Compact 2-Slice Toaster: Kitchen & Dining'):
    print tree,dir(tree)

(S
  Amazon.com/NN
  :/:
  Cuisinart/NNP
  CPT-122/-NONE-
  Compact/NNP
  2-Slice/CD
  Toaster/NNP
  :/:
  Kitchen/NNP
  &/CC
  Dining/NNP) ['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__delslice__', '__dict__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getslice__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__setslice__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', '_frozen_class', '_get_node', '_label', '_parse_error', '_pformat_flat', '_repr_png_', '_set_node', 'append', 'chomsky_normal_form', 'collapse_unary', 'convert', 'copy', 'count', 'draw', 'extend', 'flatten', 'freeze', 'fromstring', 'height', 'index', 'insert', 'label', 'leaf_treeposition', 'leaves', 'node', 'pfor

## Reduce POS Tags

if There is a chain of nouns [ proper 'NNP'

In [6]:
webpage_pos_tags = [nlp.pos_tag(page.title) for page in webpages.values()]

In [26]:

def merge_pos_temp(temp_array):
    """ Merge tokens from the temp array into one token_tag tuple """
    
    merged_token = ' '.join(token_tag[0] for token_tag in temp_array)


    return merged_token,'NNP'

def merge_pos_tags(text_ner_tags):
    """ Combine adjacent tokens with same NER tag into one token"""
    
    merged_ner_tags = []
    temp_array = []
    
    for token_tag in text_ner_tags:
        tag = token_tag[1]
        
        
        if tag == 'NNP':
            # This gonna go to the temp
            temp_array.append(token_tag)
            
        elif tag in {'IN','-NONE-','CD'}: # continue
            # See if there something in temp, 
            if len(temp_array) != 0:
                #    if yes add it to it
                temp_array.append(token_tag)
            else:
                #    if no add this to result
                merged_ner_tags.append(token_tag)
        else: 
            # Empty the temp
            if len(temp_array) != 0:
                merged_token,merged_tag = merge_pos_temp(temp_array)
                merged_ner_tags.append((merged_token,merged_tag))
                temp_array = []
            
            # add this to the result
            merged_ner_tags.append(token_tag)
        
        #print token_tag,tag,temp_array,merged_ner_tags
        
    if len(temp_array) != 0:
        merged_token,merged_tag = merge_pos_temp(temp_array)
        merged_ner_tags.append((merged_token,merged_tag))
    return merged_ner_tags

In [27]:
for i in range(len(webpage_pos_tags)):
    print merge_pos_tags(webpage_pos_tags[i])

[('Georgia Institute of Technology College of Computing', 'NNP'), ('-', ':'), ('Wikipedia', 'NNP'), (',', ','), ('the', 'DT'), ('free', 'JJ'), ('encyclopedia', 'NN')]
[('F.C', 'JJ'), ('.', '.'), ('Barcelona News', 'NNP'), ('and', 'CC'), ('Discussion /r/Barca', 'NNP')]
[('I', 'PRP'), ("'m", 'VBP'), ('scared', 'JJ'), ('of', 'IN'), ('Louis van Gaal', 'NNP'), ("'s", 'POS'), ("'bulldog", 'JJ'), ('face', 'NN'), ("'", "''"), (',', ','), ('admits', 'VBZ'), ('Manchester United', 'NNP'), ("'s", 'POS'), ('Marcos Rojo', 'NNP')]
[('Barca', 'NNP'), ('look', 'VBD'), ('to', 'TO'), ('continue', 'VB'), ('success', 'NN'), ('against', 'IN'), ('Atleti', 'NNP')]
[]
[('How', 'WRB'), ('to', 'TO'), ('Introduce Your Indoorsy Friend', 'NNP'), ('to', 'TO'), ('the', 'DT'), ('Outdoors', 'NNP')]
[('Amazon.com', 'NN'), (':', ':'), ('Cuisinart CPT-122 Compact 2-Slice Toaster', 'NNP'), (':', ':'), ('Kitchen', 'NNP'), ('&', 'CC'), ('Dining', 'NNP')]
[('Man', 'NN'), ('behind', 'IN'), ('NSA', 'NNP'), ('leaks', 'NNS'), ('s

## Combining it
```
Input :  text
Output: List of token-tag tuples

Returns pos tags with combined Proper Nouns


Idea : The proper nouns present in the title are a very strong indicator of what is there in the text
But so are the other nouns,... Running out of time for adding support for all nouns
```

In [30]:
def reduced_pos_tag(text):
    """ Generate POS tags and combine proper nouns
    
    Input :  text
    Output: List of token-tag tuples

    Returns pos tags with combined Proper Nouns


    Idea : The proper nouns present in the title are a very strong indicator of what is there in the text
    But so are the other nouns,... Running out of time for adding support for all nouns
    
    """
    text_pos_tags = nlp.pos_tag(text)
    return merge_pos_tags(text_pos_tags)
    

In [32]:
for i in range(len(webpages)):
    print reduced_pos_tag(webpages.values()[i].title)

[('Georgia Institute of Technology College of Computing', 'NNP'), ('-', ':'), ('Wikipedia', 'NNP'), (',', ','), ('the', 'DT'), ('free', 'JJ'), ('encyclopedia', 'NN')]
[('F.C', 'JJ'), ('.', '.'), ('Barcelona News', 'NNP'), ('and', 'CC'), ('Discussion /r/Barca', 'NNP')]
[('I', 'PRP'), ("'m", 'VBP'), ('scared', 'JJ'), ('of', 'IN'), ('Louis van Gaal', 'NNP'), ("'s", 'POS'), ("'bulldog", 'JJ'), ('face', 'NN'), ("'", "''"), (',', ','), ('admits', 'VBZ'), ('Manchester United', 'NNP'), ("'s", 'POS'), ('Marcos Rojo', 'NNP')]
[('Barca', 'NNP'), ('look', 'VBD'), ('to', 'TO'), ('continue', 'VB'), ('success', 'NN'), ('against', 'IN'), ('Atleti', 'NNP')]
[]
[('How', 'WRB'), ('to', 'TO'), ('Introduce Your Indoorsy Friend', 'NNP'), ('to', 'TO'), ('the', 'DT'), ('Outdoors', 'NNP')]
[('Amazon.com', 'NN'), (':', ':'), ('Cuisinart CPT-122 Compact 2-Slice Toaster', 'NNP'), (':', ':'), ('Kitchen', 'NNP'), ('&', 'CC'), ('Dining', 'NNP')]
[('Man', 'NN'), ('behind', 'IN'), ('NSA', 'NNP'), ('leaks', 'NNS'), ('s

In [33]:
for i in range(len(webpages)):
    print reduced_pos_tag(webpages.values()[i].meta_description)

[]
[('reddit', 'NN'), (':', ':'), ('the', 'DT'), ('front', 'NN'), ('page', 'NN'), ('of', 'IN'), ('the', 'DT'), ('internet', 'NN')]
[('Manchester United', 'NNP'), ('defender', 'NN'), ('Marcos Rojo', 'NNP'), ('admits', 'VBZ'), ('hes', 'NNS'), ('scared', 'VBN'), ('of', 'IN'), ('Louis van Gaals', 'NNP'), ('bulldog', 'NN'), ('face', 'NN'), ('.', '.')]
[('<', 'NN'), ('p', 'NN'), ('>', ':'), ('Atletico Madrid', 'NNP'), ('and', 'CC'), ('Barcelona', 'NNP'), ('will', 'MD'), ('meet', 'VB'), ('in', 'IN'), ('an', 'DT'), ('early-season', 'NN'), ('titan', 'NN'), ('clash', 'NN'), (',', ','), ('one', 'CD'), ('that', 'WDT'), ('ESPN FC', 'NNP'), ("'s", 'POS'), ('Gab Marcotti', 'NNP'), ('feels', 'VBZ'), ('is', 'VBZ'), ('not', 'RB'), ('a', 'DT'), ('make', 'NN'), ('or', 'CC'), ('break', 'VB'), ('matchup. < /p >', 'NNP')]
[]
[('Share', 'NN'), ('your', 'PRP$'), ('passion', 'NN'), ('for', 'IN'), ('the', 'DT'), ('outdoors', 'NNS'), ('and', 'CC'), ('introduce', 'VB'), ('a', 'DT'), ('friend', 'NN'), ('to', 'TO'),

__Shipped it to nlp module__

Testing


In [5]:
for i in range(len(webpages)):
    print [x for x,y in nlp.reduced_pos_tag(webpages.values()[i].title) if y == 'NNP']

['Georgia Institute of Technology College of Computing', 'Wikipedia']
['Barcelona News', 'Discussion /r/Barca']
['Louis van Gaal', 'Manchester United', 'Marcos Rojo']
['Barca', 'Atleti']
[]
['Introduce Your Indoorsy Friend', 'Outdoors']
['Cuisinart CPT-122 Compact 2-Slice Toaster', 'Kitchen', 'Dining']
['NSA']
